In [3]:
# #Numpy - for scientific computing
import numpy as np
# import scipy

# #For image reading and analysis
import glob
# import skimage.io
# import skimage.filters as sf
# from skimage import io, measure, filters, transform, morphology

# #Matplotlib plotting packages
import matplotlib as mpl
import matplotlib.pyplot as plt


# For loading bars
from tqdm.notebook import tqdm as tqdm

# # For creating DataFrames
import pandas as pd

In [4]:
import gliding_pkg.gliding_anal as gpkg

In [5]:
def expparam_adder(df_good, param_df):
    zeros = np.zeros(len(df_good))
    df_good['Motor Type'] = [param_df.loc['Motor_Type']['Value']]*len(df_good)
    df_good['Motor_Conc_uM'] = zeros + float(param_df.loc['Motor_Conc_uM']['Value'])
    df_good['MT_Conc_uM'] = zeros + float(param_df.loc['MT_Conc_uM']['Value'])
    df_good['ATP_Conc_uM'] = zeros + float(param_df.loc['ATP_Conc_uM']['Value'])
    df_good['ADP_Conc_uM'] = zeros + float(param_df.loc['ADP_Conc_uM']['Value'])
    df_good['P_Conc_uM'] = zeros + float(param_df.loc['P_Conc_uM']['Value'])
    df_good['Flag_Conc_mg-ml'] = zeros + float(param_df.loc['Flag_Conc_mg-ml']['Value'])
    df_good['BSA_Caesin_Both'] = [param_df.loc['BSA_Caesin_Both']['Value']]*len(df_good)
    df_good['Acq_Time_s'] = zeros + float(param_df.loc['Acq_Time_s']['Value'])
    df_good['Time_Interval_s'] = zeros + float(param_df.loc['Time_Interval_s']['Value'])
    df_good['Cy5_exptime_ms'] = zeros + float(param_df.loc['Cy5_exptime_ms']['Value'])
    df_good['Cy5_gain'] = zeros + float(param_df.loc['Cy5_gain']['Value'])
    df_good['YFP_exptime_ms'] = zeros + float(param_df.loc['YFP_exptime_ms']['Value'])
    df_good['YFP_gain'] = zeros + float(param_df.loc['YFP_gain']['Value'])
    df_good['Temperature'] = zeros + float(param_df.loc['Temperature']['Value'])
    
    return df_good

In [594]:
file_path = '../../data/gliding/'
file_folder = '2023-11-01_Gliding/1_Changing_ADP_at_40uMATP/500uM_ADP'

In [595]:
param_df = pd.read_excel(file_path + file_folder + '/Exp_Params_Template.xlsx', names=['Param', 'Value']).set_index('Param')

In [596]:
files = np.sort(glob.glob(file_path + file_folder + '/TIRF*'))

In [597]:
# Params
acqu_interval = 1 #s acquisition intervals
umperpixel=0.161

In [598]:
#Lets pick 50 frames
start=0
stop=51

for file in tqdm(files):
    im_stack = gpkg.stackimport(file+'/Pos0/img*')
    im_max = np.amax(im_stack[start:stop], axis=0)
    im_thresh = gpkg.thresh_processing(im_max)
    im_label, unique_regions, region_counts = gpkg.MTregions(im_thresh) 
    im_internal = gpkg.edgeremove(im_label)
    im_sized = gpkg.smallremove(im_internal, unique_regions, region_counts)
    im_relabel = gpkg.makeskele(im_sized)
    im_cleaned = gpkg.onlylines(im_relabel)
    df = gpkg.makedf(im_cleaned, im_stack, start, stop)
    df = gpkg.speedcalculator(df, im_stack, start, stop)
    df_good = gpkg.goodtracks(df)
    df_good_MTlen = gpkg.MTlenths(df_good, im_stack, start, stop, umperpixel)
    df_final = expparam_adder(df_good_MTlen, param_df)
    #print(df_final['speed (nm/s)'][0:10])
    df_final.to_csv('../../analyzed_data/gliding/' + file.split('/')[4] + '/' +'--'.join(file.split('/')[5:]) + '_start' + str(start) + '_stop' + str(stop) + '.csv', sep=',')

  0%|          | 0/2 [00:00<?, ?it/s]

/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:245: RuntimeWarning: invalid value encountered in long_scalars
  com = np.sum(row_vals*xindex)/np.sum(row_vals)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:289: RankWarning: Polyfit may be poorly conditioned
  skele_xvals, skele_yvals, skele_params, skele_speed, skele_rsqrd = skeletonfit(kymo_thresh, pixel_size)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:233: RuntimeWarning: divide by zero encountered in double_scalars
  rsqrd = 1-(ss_res/ss_tot)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:289: RankWarning: Polyfit may be poorly conditioned
  skele_xvals, skele_yvals, skele_params, skele_speed, skele_rsqrd = skeletonfit(kymo_thresh, pixel_size)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:289: RankWarning: Polyfit may be poorly conditioned
  skele_xvals, skele_yvals, skele_params,

In [599]:
start=50
stop=100

for file in tqdm(files):
    im_stack = gpkg.stackimport(file+'/Pos0/img*')
    im_max = np.amax(im_stack[start:stop], axis=0)
    im_thresh = gpkg.thresh_processing(im_max)
    im_label, unique_regions, region_counts = gpkg.MTregions(im_thresh) 
    im_internal = gpkg.edgeremove(im_label)
    im_sized = gpkg.smallremove(im_internal, unique_regions, region_counts)
    im_relabel = gpkg.makeskele(im_sized)
    im_cleaned = gpkg.onlylines(im_relabel)
    df = gpkg.makedf(im_cleaned, im_stack, start, stop)
    df = gpkg.speedcalculator(df, im_stack, start, stop)
    df_good = gpkg.goodtracks(df)
    df_good_MTlen = gpkg.MTlenths(df_good, im_stack, start, stop, umperpixel)
    df_final = expparam_adder(df_good_MTlen, param_df)
    df_final.to_csv('../../analyzed_data/gliding/' + file.split('/')[4] + '/' +'--'.join(file.split('/')[5:]) + '_start' + str(start) + '_stop' + str(stop) + '.csv', sep=',')

  0%|          | 0/2 [00:00<?, ?it/s]

/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:245: RuntimeWarning: invalid value encountered in long_scalars
  com = np.sum(row_vals*xindex)/np.sum(row_vals)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:289: RankWarning: Polyfit may be poorly conditioned
  skele_xvals, skele_yvals, skele_params, skele_speed, skele_rsqrd = skeletonfit(kymo_thresh, pixel_size)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:233: RuntimeWarning: divide by zero encountered in double_scalars
  rsqrd = 1-(ss_res/ss_tot)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:289: RankWarning: Polyfit may be poorly conditioned
  skele_xvals, skele_yvals, skele_params, skele_speed, skele_rsqrd = skeletonfit(kymo_thresh, pixel_size)
/Users/anaduarte/RPgroup/am_atp/analysis/gliding/gliding_pkg/gliding_anal.py:353: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_good = 

In [515]:
file.split('/')[4] + '/' +'--'.join(file.split('/')[5:])

'2023-10-23_Gliding/Titrating_ATP_conc--5uM--TIRF_100x_1sInterval_1nMNCD_5uMATP_1to50Antiflag_ilid_3'